# Experiment 3: TFIDF + LDA + SVM

In [1]:
# Load for Jupyter Notebook
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('/home/elenaruiz/Documents/FNC')
import pandas as pd 
import numpy as np 
from src.utils import io

from src.fake_news_detector.core.encoders import tfidf 

## Import `raw_dataset.json`

In [2]:
articles = io.read_json_file('/home/elenaruiz/Documents/FNC/src/data/dataset_content.json')
df = pd.DataFrame(data=articles['articles']) # Put in pandas dataframe
df.head()

,adjective_words,all_joined,all_word,common_noun_words,conjunction_words,fake,negative_words,noun_phrases_words,positive_words,subtitle_word,title_adjective_words,title_common_noun_words,title_conjunction_words,title_negative_words,title_noun_phrases_words,title_positive_words,title_word,verb_words
0,"[vegetarian, make, speculate, international, h...",The Thai police have clarified to the middle A...,"[find, corpse, vegetarian, restaurant, Bangkok...","[corpse, restaurant, Bangkok, media, establish...",[brother],True,"[corpse, kill, fire, complain, deny, victim, v...","[corpse vegetarian restaurant Bangkok, make sp...",[like],"[find, make, speculate, international, media, ...",[vegetarian],"[corpse, restaurant, Bangkok]",[],[corpse],[corpse vegetarian restaurant Bangkok],[],"[find, corpse, vegetarian, restaurant, Bangkok]","[find, serve, clarify, kill, collect, complain..."
1,"[extradition, political, Swiss, priori, legal,...","The Swiss government has said Tuesday that ""a ...","[switzerland, warn, authorize, extradition, po...","[switzerland, warn, crimes, A, spokesman, gove...",[],True,"[stress, offences, offences, accuse, argue, pe...","[switzerland warn, extradition political crime...","[Justice, like, Supreme, fair, perfectly]","[spokesman, Swiss, government, clarify, case, ...","[extradition, political]","[switzerland, warn, crimes]",[],[],"[switzerland warn, extradition political crimes]",[],"[switzerland, warn, authorize, extradition, po...","[authorize, analyze, say, seem, link, A, add, ..."
2,"[navarre, educational, decide, several, great,...","The Government of Navarra, within the Skolae p...","[navarre, censor, Songs, Amaral, Shakira, song...","[censor, Songs, Amaral, Shakira, song, Madman,...",[],True,"[censor, violence, ban, bad, fight, shit]",[navarre censor Songs Amaral Shakira song Madm...,"[promote, great, hope, promote, like, affectio...","[may, use, school]",[navarre],"[censor, Songs, Amaral, Shakira, song, Madman,...",[],[censor],[navarre censor Songs Amaral Shakira song Madm...,[],"[navarre, censor, Songs, Amaral, Shakira, song...","[use, find, song, give, hope, promote, center,..."
3,"[greet, discover, Spanish, social, avoid, many...",Carmen Jiménez told her family and friends 28 ...,"[woman, pretend, blind, years, greet, people, ...","[woman, years, people, Carmen, Jiménez, family...",[],True,"[blind, blind, injury, blind, avoid]","[woman, years greet people, discover Carmen Ji...","[greet, truth, friends, truth, greet, justice]","[truth, discover]",[greet],"[woman, years, people]",[],[blind],"[woman, years greet people]",[greet],"[woman, pretend, blind, years, greet, people]","[pretend, truth, tell, tire, see, stand, say, ..."
4,"[ejaculate, last, unveil, despicable, strong, ...","Lewis Williams, a worker at an engineering fir...","[arrested, ejaculate, boss, coffee, last, four...","[boss, coffee, years, action, discussion, Lewi...",[four],True,"[arrested, detain, arrest, prison, abuse]","[ejaculate boss coffee, years, action discussi...","[United, strong, superior, proud]","[recognize, action, discussion]","[ejaculate, last]","[boss, coffee, years]",[four],[arrested],"[ejaculate boss coffee, years]",[],"[arrested, ejaculate, boss, coffee, last, four...","[arrested, recognize, detain, ejaculate, bring..."


In [3]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

## Dictionary creation and word vectorization with TF-IDF

In [4]:
from src.fake_news_detector.core.classificators import latent_analysis as la
X_train = df_train['all_word']
Y_train = df_train['fake'].values*1
X_test = df_test['all_word']
Y_test = df_test['fake'].values*1

In [5]:
from src.fake_news_detector.core.encoders.tfidf import Tfidf_model

filter_by_freq = True

tfidf_model = Tfidf_model(filter_by_freq)
df_train['tfidf_corpus'] , df_test['tfidf_corpus'] = tfidf_model.run_transformation(X_train, X_test)

In [6]:
df_train['tfidf_corpus'].head()

10     [(0, 0.11165739703987086), (1, 0.1116573970398...
130    [(8, 0.04402014274075479), (19, 0.045961727296...
51     [(26, 0.01214542096980436), (55, 0.01414426029...
11     [(26, 0.01811468164467784), (29, 0.01148471282...
114    [(22, 0.04274708832295291), (31, 0.01893911313...
Name: tfidf_corpus, dtype: object

## Topic modeling transformation with LDA

In [7]:
from src.fake_news_detector.core.encoders.lda import LDA_model

In [8]:
train_data = df_train['tfidf_corpus']
test_data = df_test['tfidf_corpus']
n_topics = 10
dictionary = tfidf_model.get_dictionary()

lda_model = LDA_model(dictionary, n_topics)
df_train['lda_corpus'], df_test['lda_corpus'] = lda_model.run_modeling(train_data, test_data)

/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


In [9]:
lda_model.print_topics()

Topic: 0 
Words: 0.001*"BBVA" + 0.001*"restaurant" + 0.001*"songs" + 0.001*"purchase" + 0.001*"Villarejo" + 0.001*"forest" + 0.001*"use" + 0.001*"Melilla" + 0.001*"listen" + 0.001*"military"
Topic: 1 
Words: 0.001*"rice" + 0.001*"cheese" + 0.001*"Switzerland" + 0.001*"Ikea" + 0.001*"political" + 0.001*"bathroom" + 0.001*"store" + 0.001*"ugly" + 0.001*"cents" + 0.001*"use"
Topic: 2 
Words: 0.001*"Bitcoin" + 0.001*"Echenique" + 0.001*"script" + 0.001*"Vallecas" + 0.001*"Washington" + 0.001*"panda" + 0.001*"Hathaway" + 0.001*"Wilson" + 0.001*"license" + 0.001*"OT"
Topic: 3 
Words: 0.001*"Vox" + 0.001*"Abascal" + 0.001*"prisoners" + 0.001*"manicure" + 0.001*"foreign" + 0.001*"Cs" + 0.001*"Maroto" + 0.001*"PP" + 0.001*"Rivera" + 0.001*"waistcoats"
Topic: 4 
Words: 0.001*"dog" + 0.001*"guitar" + 0.001*"feminism" + 0.001*"Guindos" + 0.001*"women" + 0.001*"Oxford" + 0.001*"men" + 0.001*"Cuba" + 0.001*"flag" + 0.001*"Catalonia"
Topic: 5 
Words: 0.001*"Cross" + 0.001*"refugees" + 0.001*"Red" + 0

## Classification with SVM

In [10]:
from src.fake_news_detector.core.classificators import SupportVectorMachine as s
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [11]:
def svc_param_selection(X, y, nfolds, kernel):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel=kernel), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [12]:
X_train_lda = np.array(list(map(np.array, df_train.lda_corpus)))
X_test_lda = np.array(list(map(np.array, df_test.lda_corpus)))

In [13]:
print('For rbf:', svc_param_selection(X_train_lda, Y_train, 2, 'rbf'))
print('For linear:', svc_param_selection(X_train_lda, Y_train, 2, 'linear'))
print('For poly:', svc_param_selection(X_train_lda, Y_train, 2, 'poly'))
print('For sigmoid:', svc_param_selection(X_train_lda, Y_train, 2, 'sigmoid'))

/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


For rbf: {'C': 10, 'gamma': 0.01}


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


For linear: {'C': 0.001, 'gamma': 0.001}


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


For poly: {'C': 0.001, 'gamma': 0.001}
For sigmoid: {'C': 0.001, 'gamma': 0.001}


/home/elenaruiz/Documents/FNC/env/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [16]:
models = {}
models['rbf'] = svm.SVC(kernel='rbf', C= 10, gamma=0.1)
models['linear']  = svm.SVC(kernel='linear', C= 0.001, gamma=0.001)
models['poly']  = svm.SVC(kernel='poly',C= 0.001, gamma=0.001)
models['sigmoid'] = svm.SVC(kernel='sigmoid', C=0.001, gamma=0.001)

In [17]:
scores = s.run_models(models, X_train_lda, Y_train, X_test_lda, Y_test)
for model in scores:
    print('For model', model)
    print('Training score: {}. Test score: {}'.format(scores[model]['train'],scores[model]['test']))

For model rbf
Training score: 0.6146788990825688. Test score: 0.48148148148148145
For model linear
Training score: 0.5229357798165137. Test score: 0.4642857142857143
For model poly
Training score: 0.5229357798165137. Test score: 0.4642857142857143
For model sigmoid
Training score: 0.5229357798165137. Test score: 0.4642857142857143
